# Final Project: PanClassif

Import libraries

In [1]:
# import panclassif as pc
import pandas as pd
import numpy as np
from panclassif import dataProcess, featSelect, upsampled, binary_merge, multi_merge, classification
from sklearn.ensemble import RandomForestClassifier

### Feature Selection

In [2]:
homepath_rare = './rare_data'
cancerpath_rare = './rare_data/cancer'
normalpath_rare = './rare_data/normal'

In [3]:
names_rare = featSelect(homepath_rare,cancerpath_rare,normalpath_rare, k=7)

Cancer files: ['SARC.csv.gz', 'PCPG.csv.gz', 'THYM.csv.gz', 'KICH.csv.gz', 'CHOL.csv.gz']
Normal files: ['PCPG.norm.csv.gz', 'THYM.norm.csv.gz', 'CHOL.norm.csv.gz', 'KICH.norm.csv.gz', 'SARC.norm.csv.gz']
Feature selection process is running...
Processing file index: 0
Doing feature selection for CHOL
Processing file index: 1
Doing feature selection for KICH
Processing file index: 2
Doing feature selection for PCPG
Processing file index: 3
Doing feature selection for SARC
Processing file index: 4
Doing feature selection for THYM
Feature selection process has ended



In [4]:
names_rare

['CHOL', 'KICH', 'PCPG', 'SARC', 'THYM']

In [5]:
# homepath_common = './common_data/'
# cancerpath_common = './common_data/cancer'
# normalpath_common = './common_data/normal'

In [6]:
# names_common = featSelect(homepath_common,cancerpath_common,normalpath_common, k=5)

In [7]:
# names_common

### Data Process

In [8]:
smoothed_cancer_rare = './rare_data/cancer_smoothed'
smoothed_normal_rare = './rare_data/normal_smoothed'

In [9]:
dataProcess(homepath=homepath_rare,
            names=names_rare,
            cancerpath=cancerpath_rare,
            smoothed_cancer=smoothed_cancer_rare,
            smoothed_normal=smoothed_normal_rare,
            scale_mode=0,
            cancer='CHOL')

Data process is running
Unique genes: 35
CHOL
(45, 35)
(9, 35)
KICH
(91, 35)
(25, 35)
PCPG
(187, 35)
(3, 35)
SARC
(265, 35)
(2, 35)
THYM
(122, 35)
(2, 35)
Data process has ended



In [10]:
# smoothed_cancer_common = './common_data/cancer_smoothed'
# smoothed_normal_common = './common_data/normal_smoothed'

In [11]:
# dataProcess(homepath=homepath_common,
#             names=names_common,
#             cancerpath=cancerpath_common,
#             smoothed_cancer=smoothed_cancer_common,
#             smoothed_normal=smoothed_normal_common,
#             scale_mode=0,
#             cancer='BRCA')

Upsampling

In [12]:
upsampled(names_rare, homepath_rare)

Upsampling on train data is running
Upsampling on train data has ended



In [13]:
# upsampled(names_common, homepath_common)

Binary merge

In [14]:
binary_merge(names_rare, homepath_rare)

Merging for binary has started
Merging for binary has ended



In [15]:
# binary_merge(names_common, homepath_common)

Multi-merge

In [16]:
multi_merge(names_rare, homepath_rare)

Merging for multiclass has started
Merging for multiclass has ended



In [17]:
# multi_merge(names_common, homepath_common)

Binary Classification

In [18]:
# classification(homepath_rare, RandomForestClassifier(n_estimators=100), mode="binary", save_model="RF")

In [19]:
# classification(homepath_common, RandomForestClassifier(n_estimators=100), mode="binary", save_model="RF")

Multi-classification

In [20]:
# classification(homepath_rare, RandomForestClassifier(n_estimators=100), mode="multi", save_model="RF")

In [21]:
# classification(homepath_common, RandomForestClassifier(n_estimators=100), mode="multi", save_model="RF")

## Try ncv with rare dataset

In [23]:
from panclassif.ncvclass import NestedCV

In [37]:
#train data load
Cancer = pd.read_csv(homepath_rare+"/train_data/bin_Cancer.txt.bz2",header=None, delimiter = "\t")
Normal = pd.read_csv(homepath_rare+"/train_data/bin_Normal.txt.bz2",header=None, delimiter = "\t")
Cancer['Target'] = 1
Normal['Target'] = 0
#Normal = Normal.drop(Normal.index[0])
frame = [Cancer,Normal]
Data = pd.concat(frame,axis=0)
X_train = Data.iloc[:,:len(Data.columns)-1]
y_train = Data.iloc[:,len(Data.columns)-1]

In [51]:
# Import libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import optuna
import optuna.visualization as vis

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

# Define objective function
def objective(trial):
    # Suggest values for hyperparameters
    n_estimators = trial.suggest_int("n_estimators", 10, 500, log=True)
    max_depth = trial.suggest_int("max_depth", 2, 32)
    min_samples_split = trial.suggest_int("min_samples_split", 2, 10)
    min_samples_leaf = trial.suggest_int("min_samples_leaf", 1, 10)

    # Create and fit random forest model
    model = RandomForestRegressor(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        random_state=42,
    )
    model.fit(X_train, y_train)

    # Make predictions and calculate RMSE
    y_pred = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    # Return MAE
    return rmse

# Create study object
study = optuna.create_study(direction="minimize")

# select median rmse

# create dictionary to store all scores and trial infos (k + scores)
# plots: for inner loops --> RMSE select median
# plots: for outer loops --> MCC select best

# Run optimization process
study.optimize(objective, n_trials=50, show_progress_bar=True)

# Print best trial and best hyperparameters
print("Best trial:", study.best_trial)
print("Best hyperparameters:", study.best_params)


  0%|          | 0/50 [00:00<?, ?it/s]

Best trial: FrozenTrial(number=48, state=1, values=[0.022828282828282826], datetime_start=datetime.datetime(2024, 7, 3, 14, 26, 7, 23878), datetime_complete=datetime.datetime(2024, 7, 3, 14, 26, 7, 88153), params={'n_estimators': 22, 'max_depth': 13, 'min_samples_split': 3, 'min_samples_leaf': 1}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'n_estimators': IntDistribution(high=500, log=True, low=10, step=1), 'max_depth': IntDistribution(high=32, log=False, low=2, step=1), 'min_samples_split': IntDistribution(high=10, log=False, low=2, step=1), 'min_samples_leaf': IntDistribution(high=10, log=False, low=1, step=1)}, trial_id=48, value=None)
Best hyperparameters: {'n_estimators': 22, 'max_depth': 13, 'min_samples_split': 3, 'min_samples_leaf': 1}


In [52]:
# Visualize optimization history
vis.plot_optimization_history(study)

In [53]:
# Visualize parameter importance
vis.plot_param_importances(study)

In [54]:
# Visualize slice plot
vis.plot_slice(study, params=["n_estimators", "max_depth"])

In [55]:
# Visualize contour plot
vis.plot_contour(study, params=["min_samples_split", "min_samples_leaf"])

In [56]:
# Visualize parallel coordinate plot
vis.plot_parallel_coordinate(study)